In [413]:
import boto3
import pandas as pd
from pprint import pprint
import json

In [414]:
secrets = pd.read_json("../secrets/credentials.json")
creds = {"aws_access_key_id": secrets["aws"]["key_id"],
       "aws_secret_access_key": secrets["aws"]["key"]
}

client = boto3.client("kinesis",**creds)

stream_name = "finance"

In [415]:
stream = client.describe_stream(StreamName=stream_name)
pprint(stream)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DescribeStream operation: Stream finance_single under account 324805955411 not found.

In [ ]:
def get_shard_iterator(shard_id):
    shard_iterator = client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType="TRIM_HORIZON"
    )
    return shard_iterator["ShardIterator"]
    
def get_records(shard_id):
    iterator = get_shard_iterator(shard_id)
    records = client.get_records(
        ShardIterator=iterator,
        Limit=5000
    )
    df = pd.DataFrame(records["Records"])
    return df

In [ ]:
shards = ["000000000000"]
shard_data = map(lambda s: get_records(s), shards)

df = pd.concat(shard_data)
df.groupby("PartitionKey").count()

,SequenceNumber,ApproximateArrivalTimestamp,Data
PartitionKey,,,
thau-test,2,2,2
thau-test-2,3,3,3
thau-test-3,2,2,2
